## Matrix Completion
We aim to complete a matrix of incomplete movie ratings with the two
matrix completion algorithms introduced in the lecture.

In [2]:
import numpy as np
import scipy.sparse as sp
%load_ext autoreload
%autoreload 2

Next, we use the provided routines that allow us to load the csv dataset and store the rankings in a matrix.

In [3]:
def read_txt(path):
    """read text file from path."""
    with open(path, "r") as path:
        return path.read().splitlines()

def load_data(dataset_path):
    """Load data in text format, one rating per line, as in the kaggle competition."""
    data = read_txt(dataset_path)[1:]
    return preprocess_data(data)

def preprocess_data(data):
    """preprocessing the text data, conversion to numerical array format."""
    def process_line(line):
        position, rating = line.split(',')
        row, column = position.split("_")
        row = row.replace("r", "")
        column = column.replace("c", "")
        return int(row), int(column), float(rating)

    def statistics(data):
        row = set([line[0] for line in data])
        column = set([line[1] for line in data])
        return min(row), max(row), min(column), max(column)

    # parse each line
    data = [process_line(line) for line in data]

    # perform basic statistics on the dataset.
    min_row, max_row, min_column, max_column = statistics(data)
    print("number of items: {}, number of users: {}".format(max_row, max_column))

    # build rating matrix.
    ratings = sp.lil_matrix((max_row, max_column))
    for row, column, rating in data:
        ratings[row - 1, column - 1] = rating
    return ratings

Finally, we load and store the data via *load_data*. Please use the path at which you have stored the csv file. Data is stored in matrix *ratings*, aka matrix $X \in \mathbb{R}^{\# movies\,x\,\# users}$. The matrix $X$ contains ratings from 1 to 5. In other words, this $1682 \times 943$ has movie ratings for 1682 movies from 943 users in the range of 1 to 5. Missing ratings are set to zero.

In [7]:
ratings = load_data('movielens100k.csv').todense()
dimensions = ratings.shape

number of items: 1682, number of users: 943


In [8]:
ratings[0:10, 0:10]

matrix([[0., 4., 0., 0., 4., 4., 0., 0., 0., 4.],
        [3., 0., 0., 0., 3., 0., 0., 0., 0., 0.],
        [4., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [3., 0., 0., 0., 0., 0., 5., 0., 0., 4.],
        [3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [5., 0., 0., 0., 0., 0., 0., 0., 5., 0.],
        [4., 0., 0., 0., 0., 2., 5., 3., 4., 4.],
        [1., 0., 0., 0., 0., 4., 5., 0., 0., 0.],
        [5., 0., 0., 0., 0., 4., 5., 0., 0., 4.],
        [3., 2., 0., 0., 0., 0., 4., 0., 0., 0.]])

Now we implement projection $P_\Omega : \mathbb{R}^{\#movies\,x\,\#users}\longrightarrow \mathbb{R}^{\# known\, entries}$ that stores all entries of $X$ in a vector, and its transpose operation $P_{\Omega}^T$.

In [10]:
def projection(matrix, known_indices):
    known_entries = matrix.flat[known_indices]
    return known_entries

def transpose_projection(known_entries, known_indices, matrix_dimensions):
    matrix = np.zeros(matrix_dimensions)
    matrix.flat[known_indices] = known_entries
    return matrix

Turn for proximal map for the nuclear norm, i.e.
$$(I+\gamma \partial\|\cdot\|_*)^{-1}(Z)= \underset{X\in\mathbb{R}^{\#movies\,x\,\#users}}{\arg\,min}\left\{ frac{1}{2}\|X-Z\|_{Fro}^2+\gamma\|X\|_*  \right\}=U\,diag\,(soft_{\gamma}(\sigma))V^T $$
\
where $Z=U\,diag(\sigma)V^T$ and $\sigma=\sigma_1,\sigma_2,...,\sigma_{943})$ is a vectorof singular values, and $soft_{\gamma}$ dentotes the point-wise soft-thresholding operator, i.e.
<br>
\
$$
soft_{\gamma}(\sigma)_j=\begin{cases}
    \sigma_j-\gamma, & \text{if $\sigma_j > \gamma$}.\\
    0, & \text{otherwise}.
  \end{cases}
$$

In [12]:
def soft_shrinkage(input_vector, regularisation_parameter=1):
    return np.clip(input_vector - regularisation_parameter, a_min=0, a_max=np.inf)

def nuclear_norm_proximal_mapping(matrix, regularisation_parameter=1):
    U, S, V_transpose = np.linalg.svd(matrix, full_matrices=0)
    return U @ np.diag(soft_shrinkage(S, regularisation_parameter)) @ V_transpose

In [13]:
known_indices = np.where(ratings.ravel() != 0)[1]
known_ratings = projection(ratings, known_indices)
known_ratings_centred = known_ratings - 2.5

Now we can implement non-accelerated and accelerated matrix completion to complete missing entries of the matrix

In [15]:
def matrix_completion(known_indices, known_entries, matrix_dimensions, scaling_parameter=10, \
                      maximum_no_iterations=5000, tolerance=10**(-8), acceleration=False):
        
    low_rank_matrix = np.zeros(matrix_dimensions)
    entries = known_entries
    previous_entries = known_entries
    
    step_size = 1
    adaptive_step_size = 0
    counter = 0
    sensitivity = 1/2 * np.linalg.norm(known_entries) ** 2
    sensitivities = []
    sensitivities.append(sensitivity)
    
    while (sensitivity > tolerance) and (counter < maximum_no_iterations):
        argument = (1 + adaptive_step_size) * entries - adaptive_step_size * previous_entries
        
        low_rank_matrix = nuclear_norm_proximal_mapping(step_size * transpose_projection(argument, \
                                                            known_indices, matrix_dimensions), scaling_parameter)
        
        previous_entries = entries
        
        residual = projection(low_rank_matrix, known_indices) - known_entries
        
        entries = argument - residual
        
        sensitivity = 1/2 * np.linalg.norm(residual) ** 2
        sensitivities.append(sensitivity)
        counter += 1
        
        if counter % 50 == 0:
            print('Iteration [%d/%d], sensitivity: %.4f/%.4f' 
                   %(counter, maximum_no_iterations, sensitivity, tolerance))
            
        if acceleration == True:
            adaptive_step_size = (counter - 1)/(counter + 3)
            
    print('Completed after iteration [%d/%d], sensitivity: %.4f/%.4f' 
                   %(counter, maximum_no_iterations, sensitivity, tolerance))
        
    return low_rank_matrix, sensitivities

In [16]:
complete_ratings_1, _ = matrix_completion(known_indices, known_ratings, dimensions, \
                                        scaling_parameter=100, acceleration=True)

Iteration [50/5000], sensitivity: 0.0050/0.0000
Iteration [100/5000], sensitivity: 0.0000/0.0000
Iteration [150/5000], sensitivity: 0.0000/0.0000
Iteration [200/5000], sensitivity: 0.0000/0.0000
Completed after iteration [203/5000], sensitivity: 0.0000/0.0000


This time subtract the value 2.5 from your known movie
ratings, and add it back once you’ve completed the matrix completion.

In [17]:
complete_ratings_2, _ = matrix_completion(known_indices, known_ratings_centred, dimensions, \
                                        scaling_parameter=100, acceleration=True)
complete_ratings_2 += 2.5

Iteration [50/5000], sensitivity: 0.0360/0.0000
Iteration [100/5000], sensitivity: 0.0003/0.0000
Iteration [150/5000], sensitivity: 0.0000/0.0000
Iteration [200/5000], sensitivity: 0.0000/0.0000
Iteration [250/5000], sensitivity: 0.0000/0.0000
Iteration [300/5000], sensitivity: 0.0000/0.0000
Completed after iteration [315/5000], sensitivity: 0.0000/0.0000


Results comparision

In [18]:
np.set_printoptions(precision=1)
print(ratings[0:8, 0:8], '\n\n', complete_ratings_1[0:8, 0:8], '\n\n', complete_ratings_2[0:8, 0:8])

[[0. 4. 0. 0. 4. 4. 0. 0.]
 [3. 0. 0. 0. 3. 0. 0. 0.]
 [4. 0. 0. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0. 0. 5. 0.]
 [3. 0. 0. 0. 0. 0. 0. 0.]
 [5. 0. 0. 0. 0. 0. 0. 0.]
 [4. 0. 0. 0. 0. 2. 5. 3.]
 [1. 0. 0. 0. 0. 4. 5. 0.]] 

 [[ 2.3e+00  4.0e+00  2.1e-01  3.8e-01  4.0e+00  4.0e+00  2.4e+00  1.8e+00]
 [ 3.0e+00  3.4e-01 -1.3e-01 -1.4e-01  3.0e+00  5.9e-01  1.7e+00  2.0e+00]
 [ 4.0e+00  4.3e-01 -1.2e-01  1.5e-01  1.3e-01  2.7e-01 -5.4e-02  9.9e-02]
 [ 3.0e+00 -4.5e-02 -4.9e-04 -1.2e-01  1.4e+00  1.7e+00  5.0e+00  2.0e+00]
 [ 3.0e+00  2.7e-01  9.6e-02  1.5e-01  7.7e-01  7.7e-01  1.6e+00  3.4e-01]
 [ 5.0e+00  5.0e-01 -4.7e-01 -1.9e-01  5.0e-02  1.6e-01  7.0e-01 -8.1e-01]
 [ 4.0e+00  1.2e+00  4.2e-01  5.8e-01  3.1e+00  2.0e+00  5.0e+00  3.0e+00]
 [ 1.0e+00  6.9e-01  5.9e-02 -4.7e-02  1.5e+00  4.0e+00  5.0e+00  1.5e+00]] 

 [[3.3 4.  2.5 2.7 4.  4.  3.6 3.4]
 [3.  2.7 2.6 2.6 3.  2.5 3.2 3. ]
 [4.  2.5 2.4 2.5 2.4 2.4 2.2 2.7]
 [3.  2.7 2.3 2.4 3.  3.4 5.  3. ]
 [3.  2.9 2.3 2.5 2.7 3.3 3.  2.8]
 

In [19]:
print('Rank of non-centred matrix is %d. \n\n Rank of centred matrix is %d.'
      %(np.linalg.matrix_rank(complete_ratings_1), np.linalg.matrix_rank(complete_ratings_2 - 2.5)))

Rank of non-centred matrix is 385. 

 Rank of centred matrix is 321.
